In [130]:
'''
Zaobserwowane błędy:
- jeden token nie ma żadnej annotation
- występują po dwie relacje które opisują dokładnie to samo, a mają różne ID - duplikaty
- 17 tokenów nie zawiera żadnych relacji - ? sprawdzić ? - kilka takich problemów w obrębie jednego pliku
- gdyby jednak subject->action, to dwóm brakuje relacji
'''

'\nZaobserwowane błędy:\n- jeden token nie ma żadnej annotation\n- występują po dwie relacje które opisują dokładnie to samo, a mają różne ID - duplikaty\n- 17 tokenów nie zawiera żadnych relacji - ? sprawdzić ? - kilka takich problemów w obrębie jednego pliku\n- gdyby jednak subject->action, to dwóm brakuje relacji\n'

In [1]:
import re
import os

In [75]:
root = '/home/jells123/Documents/ENGINEER/DATA/1-Raw data/data/'
ann = '/annotations'

sets = {
    'Train' : 'train',
    'Dev' : 'dev',
    'Test' : 'test_3'
}
set_type = 'Test'
path = root + sets[set_type] + ann

rel_pattern = r'(?P<r_id>R[0-9]+)\s+[A-Za-z]+\s+(?P<r_first>[A-Za-z]+:T[0-9]+)\s+(?P<r_second>[A-Za-z]+:T[0-9]+)'
ann_pattern = r'(?P<a_id>A[0-9]+)\s+(?P<cat>[A-Za-z]+)\s+(?P<t_id>T[0-9]+)\s+(?P<text>[^\n]+)'
token_pattern = r'(?P<t_id>T[0-9]+)\s+(?P<t_type>[A-Z][a-z]+) (?P<indices>[0-9]+ [0-9]+[^\t]+)\t(?P<t_text>.+)\n'

In [3]:
main_cats = ['ActionName', 'Capability', 'StrategicObjectives', 'TacticalObjectives']
# for each main category, store token groups that wasn't annotated using this main category

def init_data():
    data = dict([(cat, {}) for cat in main_cats])
    data['ActionName'] = {
        'File' : [],
        'Network' : [],
        'Other' : []
    }
    data['Capability'] = {
        'command_and_control' : [],
        'infection/propagation' : [],
        'other' : []
    }
    undefined = dict([(cat, []) for cat in main_cats])
    return data, undefined

In [4]:
def process_annotation(anns, words, token, cat):
    
    # say cat == 'ActionName'
    action_name = list(filter(lambda x : x[1] == cat, anns))
    if action_name:
        m = re.search(r'(?P<id>[0-9]{3}):(?P<type>[A-Za-z]+)-(?P<name>.+)\s*', action_name[0][3])
        typ, name = m.group('type'), m.group('name')
        
        if cat == 'Capability':
            # special treatment :)
            if name in data[cat].keys():
                data[cat][name].append(words)
            else:
                data[cat]['other'].append(words)
        else:
            if typ in data[cat].keys():
                data[cat][typ].append(words)
            else:
                data[cat]['Other'].append(words)
            
    else:
        undefined[cat].append(words)
        

In [39]:
def get_token_words_relation(t, tokens, rels, anns, cat):
    
    global tokens_count
    tokens_count += 1
    
    relations = list(filter(lambda x : x[1] == 'Action:'+t[0] , rels))
    if relations:
        mod_relations = list(filter(lambda x : 'Modifier' in x[2], relations))
        if mod_relations:
            for mr in mod_relations:
                relations += list(filter(lambda x : x[1] == mr[2], rels))
    
    if include_subject:
        subject = list(filter(lambda x : 'Subject' in x[1] and x[2] == 'Action:'+t[0], rels))
        if subject:
            subject = subject[0]
            relations.append(subject)

    if not relations:
        print("Token {} has no relation.".format(t))
        global relation_missing
        relation_missing += 1
    
    words = []
    words.append(t[3])
    
    for r in relations:
        
        if include_subject and 'Subject' in r[1]: 
            # subject -> action
            token_id = r[1].split(':')[1]
            token_word = list(filter(lambda x : x[0] == token_id, tokens))
            if token_word and token_word[0][3] not in words:
                words.insert(0, token_word[0][3])
            
        else:
            # action -> object, action -> mod -> object
            token_id = r[2].split(':')[1]
            token_word = list(filter(lambda x : x[0] == token_id, tokens))
            if token_word and token_word[0][3] not in words:
                if token_word[0][3][0].isupper():
                    words.insert(0, token_word[0][3])
                else:
                    words.append(token_word[0][3])
    
    # relation gathered and here we create the dataset
    # id prefer it in anotehr function but whatever
    annotations = list(filter(lambda x : x[2] == t[0], anns))
    process_annotation(annotations, words, t, cat)
    
    if not annotations:
        print("Token {} has no annotation.".format(t))
        global annotation_missing
        annotation_missing += 1
                


In [7]:
def get_token_words_coords(t, text, off=0):
    indices = t[2]
    indices = indices.split(';') if ';' in indices else [indices]
    real_token = ''
    indices = [[int(idx) for idx in i.split()] for i in indices]
    for i in indices:
        real_token += text[i[0]+off : i[1]+off] + ' '
        
    while '- ' in real_token:
        real_token = real_token.replace('- ', '')
    return real_token.strip(), (indices[0][0]+off, indices[-1][1]+off)

def get_token_neighbors(token, coords, text, neighbors):
    
    left = (neighbors-1)/2
    right = left
    
    l_idx = coords[0] - 1
    r_idx = coords[1] + 1
    
    words = []
    while left:
        while text[l_idx].isspace():
            l_idx -= 1
        word = ''
        while not text[l_idx].isspace():
            word += text[l_idx]
            l_idx -= 1
        
        word = word[::-1]
        if word[0] == '<' or word[-1] == '>': #html tag
            l_idx -=1
            word = ''
        else:
            words.insert(0, word)
            left -= 1
    
    words.append(token)
    while right:
        while text[r_idx].isspace():
            r_idx += 1
        word = ''
        while not text[r_idx].isspace():
            word += text[r_idx]
            r_idx += 1
            
        if word[0] == '<' or word[-1] == '>':
            r_idx += 1
            word = ''
        else:
            words.append(word)
            right -= 1
            
    return words
        
def gather_data(words, anns, t, cat):
    annotations = list(filter(lambda x : x[2] == t[0], anns))
    process_annotation(annotations, words, t, cat)
    
    if not annotations:
        print("Token {} has no annotation.".format(t))
        global annotation_missing
        annotation_missing += 1

In [79]:
data, undefined = init_data()
print(data)

relation_mode = True
cat = 'Capability'

relation_missing = 0
annotation_missing = 0
tokens_count = 0

include_subject = True
action_tokens = []

for filename in os.listdir(path):
    
    if filename.endswith('.ann'):
        print("\n>>> {}".format(filename))
        with open(path + '/' + filename) as ann_file:
            content = ann_file.read()
            
            # match lines with a token            
            tokens = re.findall(token_pattern, content)
            relations = re.findall(rel_pattern, content)
            annotations = re.findall(ann_pattern, content)
            
            if not relation_mode:
            # read both .ann and .txt file
                with open(path + "/" + filename.replace(".ann", ".txt")) as txt_file:
                    text = txt_file.read()
                    offset = 0
                    for t in tokens:
                        if t[1] == 'Action':
                            # get token from annotations + text and compare them, are they equal?
                            ann_token = t[3]
                            tokens_count += 1
                            while '- ' in ann_token:
                                ann_token = ann_token.replace('- ', '')

                            idx_token, coords = get_token_words_coords(t, text, offset)
                            # oops, we got a mismatch!
                            if ann_token != idx_token:
                                # probably something went wrong! - we need to deal with it as always :)
                                where_is = text.find(ann_token)
                                offset = where_is - int(t[2].split()[0])
                                idx_token, coords = get_token_words_coords(t, text, offset)

                            words = get_token_neighbors(ann_token, coords, text, 9)
                            gather_data(words, annotations, t, cat)
                            
            else:
            # older version -> finds token groups by relation
                for t in tokens:
                    if t[1] == 'Action':
                        get_token_words_relation(t, tokens, relations, annotations, cat)         
                        action_tokens.append(t[3])


{'ActionName': {'File': [], 'Network': [], 'Other': []}, 'Capability': {'command_and_control': [], 'infection/propagation': [], 'other': []}, 'StrategicObjectives': {}, 'TacticalObjectives': {}}

>>> butterfly-corporate-spies-out-for-financial-gain.ann

>>> Equation_group_questions_and_answers.ann

>>> Dissecting-LinuxMoose.ann
Token ('T136', 'Action', '52031 52036', 'reach') has no annotation.

>>> Dissecting-the-Kraken.ann
Token ('T16', 'Action', '10699 10705', 'update') has no relation.
Token ('T17', 'Action', '10710 10719', 'uninstall') has no relation.

>>> DEEP_PANDA_Sakula.ann


In [9]:
temp_datasets = {}

In [10]:
import pandas as pd

'''
For main category
and its sub categories!
'''

main_cat = cat
include_undefined = False

for sub_cat in data[main_cat].keys():

    labels = []
    sentences = []
    for cat in data[main_cat].keys():
        for sent_set in data[main_cat][cat]:

            sentence = ' '.join(sent_set)

            # sentence processing here
            for punct in ["“", "”", "\[", "\]"]:
                sentence = re.sub(punct, "", sentence)

            sentences.append(sentence)

    #         labels.append(main_cat)

            if cat == sub_cat:
                labels.append(sub_cat)
            else:
                labels.append('no-' + sub_cat)

    if include_undefined:
        for sent_set in undefined[main_cat]:
            sentences.append(" ".join(sent_set))
            labels.append('no-' + sub_cat)

    sent_label = 'text-rel' if relation_mode else 'text-neigh'
            
    if sub_cat not in temp_datasets:
        dataset = pd.DataFrame({"label" : labels, sent_label : sentences})
        dataset['label_num'] = dataset.label.map({'no-'+sub_cat:0, sub_cat:1})
        temp_datasets[sub_cat] = dataset
        print(dataset.label.value_counts(), "\n")
    else:
        print(len(sentences))
        temp_datasets[sub_cat][sent_label] = sentences
        print(temp_datasets[sub_cat].label.value_counts(), "\n")

temp_datasets[sub_cat].head()

no-File    829
File       325
Name: label, dtype: int64 

no-Network    781
Network       373
Name: label, dtype: int64 

no-Other    698
Other       456
Name: label, dtype: int64 



In [30]:
for sub_cat in temp_datasets.keys():
    dset = temp_datasets[sub_cat]
    dset.to_csv('{}-{}.csv'.format(sub_cat.replace('/', '_'), set_type))

In [77]:
dataset = None

In [80]:
import pandas as pd

'''
For main category
and that's all!
'''
labels = []
sentences = []

main_cat = cat
include_undefined = False

for sub_cat in data[main_cat].keys():
    tokens_list = data[main_cat][sub_cat]
    for tokens in tokens_list:
        sentence = ' '.join(tokens)

        # sentence processing here
        for punct in ["“", "”", "\[", "\]"]:
            sentence = re.sub(punct, "", sentence)
        sentences.append(sentence)
        labels.append(main_cat)

for sent_set in undefined[main_cat]:
    sentence = ' '.join(sent_set)
    for punct in ["“", "”", "\[", "\]"]:
        sentence = re.sub(punct, "", sentence)
        
    sentences.append(sentence)
    labels.append('No' + main_cat)
   
c_label = 'text-rel' if relation_mode else 'text-neigh'
if dataset is None:
    dataset = pd.DataFrame({"label" : labels, c_label : sentences})
    dataset['label_num'] = dataset.label.map({'No'+main_cat:0, main_cat:1})
else:
    dataset[c_label] = sentences

if relation_mode:
    print(len(undefined[main_cat]))
    print(len(action_tokens))
    print()
    dataset['token'] = action_tokens
    
dataset.head()

22
110



,label,text-neigh,label_num,text-rel,token
0,Capability,a command-and-control (C&C) server used in an ...,1,used a command-and-control (C&C) server in an ...,obtaining
1,Capability,and went on to create a variant of Backdoor.Ji...,1,Bda9.tmp create a variant of Backdoor.Jiripbot...,eavesdropped
2,Capability,"air-gapped networks, they can save these comma...",1,they save these commands in the hidden area of...,insert
3,Capability,overview Linux/Moose will periodically communi...,1,Linux/Moose communicate with a set of command ...,used
4,Capability,"configuration C&C server, will provide configu...",1,the configuration C&C server provide configura...,executed


In [81]:
# dataset.to_csv('neighbours/{}-{}.csv'.format(main_cat, set_type))
dataset.to_csv('{}-{}.csv'.format(main_cat, set_type))
print(dataset.label.value_counts(), "\n")

Capability      88
NoCapability    22
Name: label, dtype: int64 

